# Travail personnel : Algorithmique et Python

BONZI Doria - M1 Industries de la langue - 11/11/2023

Ce notebook sert à traiter des fichiers afin d'afficher leurs nombres de mots et de phrases et regroupent ces statistiques dans un fichier tsv.

### Importation des modules et téléchargement de fichiers test

In [ ]:
import os
import json
import string
import re
import nltk

!pip install gdown

In [ ]:
!gdown --id 1Y6h0iynK2ZsDe5Ig5t93w-I6YbtxOidG -O LOREM.txt
!gdown --id 1JTkhHmeuS3M7zBJhTSPEFVgTEUk6escq -O MARIO.txt
!gdown --id 1tg0SfXFlIwrynyMlTh5g2vgOacsQNAFe -O DOVERBEACH.txt

In [ ]:
import nltk
nltk.download('punkt_tab')

def segment_text(input_file, output_file_sentences, output_file_words):
    with open(input_file, 'r', encoding='utf-8') as file:
        text = file.read()

    nltk.download('punkt')

    #Segmentation en phrases
    sentences = nltk.sent_tokenize(text)
    with open(output_file_sentences, 'w', encoding='utf-8') as output_sentences:
        for sentence in sentences:
            output_sentences.write(sentence + " $ " + '\n')

    #Tokénisation
    with open(output_file_sentences, 'r', encoding='utf-8') as segmented_file:
        segmented_text = segmented_file.read()
        words = nltk.word_tokenize(segmented_text)
        with open(output_file_words, 'w', encoding='utf-8') as output_words:
            for word in words:
                output_words.write(word + '\n')

segment_text('MARIO.txt', 'phrases.txt', 'mots.txt')

### Segmentation en phrases

Une fonction de segmentation en phrases (avec suppression des espaces ou retour chariot entre phrases)

```
""" fonction de segmentation en phrases
arg1 : text: le texte à segmenter
retour : une liste de phrases (string)
"""
segment(text:str)->list[str]
```



In [ ]:
exemple="""Plusieurs mois plus tard, Mia croise à nouveau Sebastian à une pool party où celui-ci, peu investi, reprend avec un groupe des succès des années 1980. Elle se moque de lui et de sa carrière stagnante ; il réplique en ironisant sur ses aspirations d'actrice. Lorsqu'ils repartent ensemble chercher leurs voitures, ils se plaignent d'avoir eu à se recroiser, mais partagent bientôt une danse devant une vue de la ville (A Lovely Night). Sebastian parvient à lui rendre visite à son café et devenus confidents, ils se font découvrir leurs centres d'intérêt respectifs : Mia lui fait part de la passion ancienne qu'elle éprouve pour les studios qu'ils visitent et plus généralement pour le cinéma ; il l'entraîne dans un club de jazz où il lui avoue son rêve d'ouvrir un jour son propre établissement. Ces moments scellent leur profonde amitié (City of Stars). Sebastian invite alors Mia à une projection de La Fureur de vivre. Elle accepte, dans un premier temps, avant de se voir rappeler au dernier moment qu'elle a déjà un rendez-vous avec Greg, son petit ami. Mais le dîner avec Greg et ses proches se révèle ennuyeux. Elle s'enfuit et court rejoindre Sebastian au cinéma, alors que la séance vient de commencer. Leur soirée s'achève par une nouvelle danse dans le planétarium de l'observatoire Griffith, aperçu dans le film qu'ils viennent de quitter ("Planetarium"). C'est là qu'ils s'embrassent pour la première fois."""

In [ ]:
def segment(text:str)->list[str]:
    sentences=[]
    current_sent=""
    for i,char in enumerate(text):
        #si on trouve une frontière de phrase
        if char==" " and (text[i-1] in "?!." and text[i+1].isupper()):

            sentences.append(current_sent)
            current_sent=""

        else:
            current_sent+=char

    if current_sent!="":
        sentences.append(current_sent)

    print("Nb de phrases:",len(sentences))
    return sentences

#texte_segmente=segment(exemple)
#print(texte_segmente)

Pour la segmentation en phrases, écrire en pseudo-code une deuxième version permettant de gérer les abréviations, à partir d'une liste définissant une liste d'abréviations à traiter :

```
Pour les abréviations, nous avons remarqué par exemple avec notre fichier LOREM.txt le traitement de deux phrases différentes avec l'abréviation "[...] de 45 av. J.-C, [...]". Nous écrivons un pseudo-code qui permettrait de gérer ce type d'abréviations.

string_abrevs ← ["M","cf","etc","ex","av", "J"...]

Fonction segmentation(texte, abrevs):
    phrases ← []
    phrase_en_cours ← ""

    Pour chaque caractère c dans le texte {
        Si caractère c est un espace " " ET est [précédé par un séparateur de phrase (point, point d'interrogation, point d'exclamation) et suivi d'une majuscule] {
            Ajouter caractère c à phrase_en_cours

            Si phrase_en_cours n'est pas vide et le mot suivant est une abréviation {
                Continuer la construction de la phrase
            } Sinon {
                Ajouter phrase_en_cours à la liste des phrases
                Réinitialiser phrase_en_cours à une chaîne vide
            } Sinon {
                Ajouter caractère c à phrase_en_cours
        }
      }
    }
    Retourner la liste des phrases
```

### Tokenisation avec suppression des espaces

Une fonction de tokenisation d'une chaînes en tokens (avec suppression des espaces) :

```
""" fonction de tokenisation
arg1 : text: le texte à tokeniser
retour : une liste de tokens (str)
"""
tokenize(text:str)->list[str]
```


In [ ]:
def tokenize(sentences:list)->list:
    prev_char=" "
    current_word=""
    tokens=[]
    nb_words=0

    for char in sentences:
        if  (char in string.punctuation+" "):
            if not (prev_char in string.punctuation+" "):
                nb_words=nb_words+1

                if char=="'":
                    current_word+=char
                tokens.append(current_word)
                current_word=""

            if char!=" " and char!="'":
                tokens.append(char)
        else:
            current_word=current_word+char

        prev_char=char

    if current_word!="":
        tokens.append(current_word)

    print("Nb de mots:",nb_words)
    return tokens

#texte_tokenise=tokenize(exemple)
#print(texte_tokenise)

In [ ]:
#Un test avec le texte d'exemple défini au début de ce notebook

#segmentation=segment(exemple)
#tokenisation=tokenize(exemple)
#print(segmentation)
#print(tokenisation)

### Lecture d'un répertoire

Une fonction de lecture d'un répertoire qui, pour chaque fichier NAME.txt crée un fichier NAME.seg.json contenant une liste de paragraphes, chaque paragraphes étant une liste de phrases, chaque phrases étant une liste de tokens. En outre, le script enregistrera dans un fichier stats.tsv les statististiques de chaque fichiers, selon les colonnes suivantes :

```
""" fonction de traitement d'un répertoire
arg1 : dir_name : le chemin vers le répertoire
retour : n le nombre fichiers traités + écriture dans le fichiers stats.tsv + écriture des fichiers NAME.seg.json
"""
process_dir(dir_name:str,stats_file_name:str)→int :
```

```
Contenu de stats.tsv :

File	        nbParas	 nbSents	nbToks	nbChars
exemple1.txt	10	     25	        221	    842
exemple2.txt	5	     10	        98	    419
```

#### [test] Sur un fichier donné

```
#On ouvre et on lit le fichier texte "LOREM", on stocke chaque ligne dans une liste "fichier".
fichier=[]

try:
    f=open('LOREM.txt', 'r')

#lecture des lignes (paragraphes)
    for ligne in f:
        fichier.append(ligne.strip())
    print(fichier)

    f.close()

except OSError as err:
    print("OS error:",err)
except ValueError:
    print("could not convert data to an integer")
except Exception as err:
    print("Unexcepted {err=}, {type(err)=}")
    raise
```

#### [test] Utilisation des fonctions définies précédemment sur un fichier donné

```
#La liste "fichier" n'est pas exploitable en tant que liste : on joint les éléments de la liste pour former un texte exploitable.
texte=" ".join(fichier)

def process_text(texte: str)->list:
    #Segmentation du texte contenu dans le fichier
    sentences=segment(texte)

    #Tokénisation de la sortie "Segmentation"
    tokens_list=[]
    for sentence in sentences:
        tokens=tokenize(sentence)
        tokens_list.append(tokens)
    return tokens_list

texte_traite=process_text(texte)
print(texte_traite)

#Création du fichier avec le texte segmenté et tokénisé
with open('texte_traite.seg.json', 'w', encoding='utf-8') as json_file:
    json.dump(texte_traite, json_file, ensure_ascii=False, indent=4)


#Création du fichier stats
with open('texte_traite_stats.tsv', 'w', encoding='utf-8') as stats_file:
    stats_file.write("Fichier\tParagraphs\tSentences\tTokens\n")
    nb_paragraphes=len(fichier)
    nb_phrases=len(texte_traite)
    nb_tokens=sum(len(sentence) for sentence in texte_traite)
    stats_file.write(f"LOREM.txt\t{nb_paragraphes}\t{nb_phrases}\t{nb_tokens}\n")
```

### Création d'une fonction pour traiter les fichiers txt d'un répertoire avec création d'un fichier seg.json et stats.tsv

In [ ]:
def process_dir(dir_name:str, stats_file_name:str)->int:

    #On ouvre le fichier de statistiques "stats.tsv"
    with open(stats_file_name, 'w', encoding='utf-8') as stats_file:
        stats_file.write("Filename\tParagraphes\tPhrases\tTokens\tCaractères\n")

        fichiers_traites=0

        #On parcourt tous les fichiers du répertoire, et si le fichier termine par ".txt", on le traite comme "input_file"
        for filename in os.listdir(dir_name):
            if filename.endswith('.txt'):
                input_file=os.path.join(dir_name, filename)

                #On lit le contenu du fichier et on le stocke dans la liste "texte"
                with open(input_file, 'r', encoding='utf-8') as file:
                    texte=file.read()

                    #Pour "nb_paragraphes", on ajoute +1 au compteur car il compte les retours à la ligne (et donc ne compte pas le premier paragrpahe)
                    nb_paragraphes=texte.count('\n')+1

                    #Cette ligne est ajoutée après avoir remarqué le traitement littéral de '\n' dans certains des fichiers.
                    texte=texte.replace('\n', ' ')

                    #On segmente et tokénise la liste texte avec les fonctions segment et tokenize
                    sentences=segment(texte)
                    tokens_list=[]
                    for sentence in sentences:
                        tokens=tokenize(sentence)
                        tokens_list.append(tokens)

                    #On crée le fichier NAME.seg.json pour "stocker" le résultat de notre segmentation et tokénisation
                    output_json_file=os.path.join(dir_name, f"{filename}.seg.json")
                    with open(output_json_file, 'w', encoding='utf-8') as json_file:
                        json.dump(tokens_list, json_file, ensure_ascii=False, indent=4)

                    #Dans le fichier stats.tsv, on écrit : nb_paragraphes, nb_phrases, nb_tokens pour chacun des fichiers dans un tableau
                    nb_phrases=len(sentences)
                    nb_tokens=sum(len(sentence) for sentence in tokens_list)
                    nb_chars=len(texte)
                    stats_file.write(f"{filename}\t{nb_paragraphes}\t{nb_phrases}\t{nb_tokens}\t{nb_chars}\n")

                    #On incrémente compteu dur des fichiers traités
                    fichiers_traites+=1
                    print(f"Le fichier {filename} a été traité.")

    #L'"output" de cette fonction sera le nombre de fichiers traités
    return fichiers_traites

#On fait la fonction process_dir sur nos données importées "/content/" ici pour ce Google Colab
process_dir("/content/","stats.tsv")